In [0]:
%sql
USE CATALOG `100cep_gateway`;

In [0]:
%sql
USE SCHEMA gold;

In [0]:
%sql
CREATE OR REPLACE TABLE `100cep_gateway`.gold.dim_clientes AS
WITH clientes AS(
SELECT 
  `100cep_gateway`.silver.`100cep_avaliacoes`cliente_id,
  cliente_id_unico,
  cep_prefixo
FROM `100cep_gateway`.silver.`100cep_clientes`
)
SELECT
  *
FROM 
  clientes

In [0]:
%sql
CREATE OR REPLACE TABLE `100cep_gateway`.gold.dim_vendedores AS
WITH vendedores AS(
SELECT 
  vendedor_id,
  cep_prefixo
FROM `100cep_gateway`.silver.`100cep_vendedores`
)
SELECT
  *
FROM
  vendedores;

In [0]:
%sql
CREATE OR REPLACE TABLE dim_pagamentos AS
WITH metodo_pagamento AS(
SELECT
  ROW_NUMBER() OVER (ORDER BY tipo_pagamento) AS id_pagamento,
  tipo_pagamento,
  nivel_risco
FROM (
  SELECT DISTINCT
    tipo_pagamento,
    CASE
      WHEN tipo_pagamento LIKE '%CREDITO%' THEN 'MEDIO'
      WHEN tipo_pagamento LIKE '%DEBITO%' THEN 'BAIXO'
      WHEN tipo_pagamento LIKE '%BOLETO%' THEN 'MEDIO'
      ELSE 'ALTO'
    END AS nivel_risco
  FROM `100cep_gateway`.silver.`100cep_pagamentos`
)
)
SELECT
  *
FROM 
  metodo_pagamento

In [0]:
%sql
CREATE OR REPLACE TABLE dim_data AS
WITH calendario AS (
  SELECT 
    DATE(data_pedido) AS data_calendario,
    DAY(data_pedido) AS dia,
    MONTH(data_pedido) AS mes,
    YEAR(data_pedido) AS ano,
    UPPER(CASE DAYOFWEEK(data_pedido)
      WHEN 1 THEN 'DOMINGO'
      WHEN 2 THEN 'SEGUNDA-FEIRA'
      WHEN 3 THEN 'TERCA-FEIRA'
      WHEN 4 THEN 'QUARTA-FEIRA'
      WHEN 5 THEN 'QUINTA-FEIRA'
      WHEN 6 THEN 'SEXTA-FEIRA'
      WHEN 7 THEN 'SABADO'
    END) AS nome_dia_semana,
    UPPER(CASE MONTH(data_pedido)
      WHEN 1 THEN 'JANEIRO'
      WHEN 2 THEN 'FEVEREIRO'
      WHEN 3 THEN 'MARÇO'
      WHEN 4 THEN 'ABRIL'
      WHEN 5 THEN 'MAIO'
      WHEN 6 THEN 'JUNHO'
      WHEN 7 THEN 'JULHO'
      WHEN 8 THEN 'AGOSTO'
      WHEN 9 THEN 'SETEMBRO'
      WHEN 10 THEN 'OUTUBRO'
      WHEN 11 THEN 'NOVEMBRO'
      WHEN 12 THEN 'DEZEMBRO'
    END) AS nome_mes
  FROM 
    `100cep_gateway`.silver.`100cep_pedidos`
  WHERE 
    data_pedido IS NOT NULL
  GROUP BY 
    DATE(data_pedido),
    DAY(data_pedido),
    MONTH(data_pedido),
    YEAR(data_pedido),
    DAYOFWEEK(data_pedido)
)
SELECT
  *
FROM 
  calendario

In [0]:
%sql
CREATE OR REPLACE TABLE dim_geolocalizacao AS 
WITH cidades_validas AS (
SELECT
  cidade,
  COUNT(cidade) AS contagens
FROM
  `100cep_gateway`.silver.`100cep_geolocalizacao`
GROUP BY
  cidade
HAVING
  contagens > 1
ORDER BY
  COUNT(cidade) DESC)

SELECT
  cep_prefixo,
  cidade,
  estado,
  latitude,
  longitude
FROM 
  `100cep_gateway`.silver.`100cep_geolocalizacao`
WHERE 
  cidade IN (
    SELECT cidade FROM cidades_validas
    )

In [0]:
%sql
CREATE OR REPLACE TABLE `100cep_gateway`.gold.fato_transacoes AS
WITH itens_agg AS (
SELECT
    pedido_id,
    SUM(preco_produto) AS preco_total,
    SUM(valor_frete) AS frete_total
  FROM 100cep_gateway.silver.`100cep_itens_pedidos`
  GROUP BY pedido_id
)

SELECT
  p.pedido_id,
  o.cliente_id,
  s.vendedor_id,
  o.data_pedido :: DATE,
  CONCAT(
      LPAD(HOUR(o.data_pedido), 2, '0'), ':',
      LPAD(MINUTE(o.data_pedido), 2, '0'), ':',
      LPAD(SECOND(o.data_pedido), 2, '0')
    ) AS horario_pedido,
  p.tipo_pagamento,
  p.valor_transacao :: DECIMAL(12,2) AS valor_transacao,
  i.preco_total,
  i.frete_total,
  o.status_pedido,
  CASE WHEN cb.chargeback_id IS NULL THEN 'NAO' ELSE 'SIM' END AS teve_chargeback,
  cb.motivo_chargeback,
  cb.status_chargeback,
  cb.resposta_emitente,
  cb.resposta_adquirente
FROM 100cep_gateway.silver.`100cep_pagamentos` AS p
LEFT JOIN 100cep_gateway.silver.`100cep_pedidos` AS o
  USING(pedido_id)
LEFT JOIN itens_agg AS i
  USING(pedido_id)
LEFT JOIN 100cep_gateway.silver.`100cep_itens_pedidos` AS oi
  USING(pedido_id)
LEFT JOIN 100cep_gateway.silver.`100cep_vendedores` AS s
  USING(vendedor_id)
LEFT JOIN `100cep_gateway`.silver.`100cep_chargebacks` AS cb
  ON i.pedido_id = cb.chargeback_id